# How to Use this Template

This template is an example for how to generate a plot using [Bokeh](http://bokeh.pydata.org). This template behaves very similarly to a normal Pinyon/Jupyter template, except that the output consists of the components of a plot. Specifically, the output should be a single variable `pinyon` (yes, overwrite the input data) which has at least the following keys:

+ scripts -> Scripts behind the bokeh plot, generated by bokeh
+ div -> Div for the plot, generated by bokeh

Other keys will be passed to the decision form template along with these variables

For more information about Bokeh plots see [the Bokeh documentation](http://bokeh.pydata.org/en/latest/docs/user_guide/embed.html).

## Input Cell (programmatically generated)

These commands connect to the pinyon server and download the inputs in pkl format into this notebook

In [1]:
import cPickle as pickle
import requests
pinyon = {}
pinyon['data'] = pickle.loads(requests.get('http://128.165.58.191:6543/tool/579107167af098579a013d24/output/data').content)
settings = {}

## Start Code Here

### Preparatory code
Getting the environment ready (e.g., creating local variables) to make the similarity engine easy

In [2]:
# Libraries
import math
import numpy as np
import pandas as pd

# Get the data out of the structure
data = pinyon['data']

# Ready the Bokeh
from bokeh.embed import components
from bokeh.layouts import column
from bokeh.models import CustomJS, ColumnDataSource, Slider, Button, HoverTool, Slider
from bokeh.plotting import Figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Making the Visualization
All the bits of work necessary to make the code, starting with defining data sources

In [3]:
# Source holding all data necessary for the plot
all_df = data.copy()
def pretty_label(x):
    if x == True:
        return 'Before'
    elif x == False:
        return 'After'
    else:
        return 'Unknown' 
def color_chooser(x):
    if x == True:
        return '#0000ff'
    elif x == False:
        return '#ff0000'
    else:
        return "#808080"
color = [ color_chooser(x) for x in data['BeforePeak'] ]
all_df['BeforePeakLabel'] = all_df['BeforePeak'].apply(pretty_label)
all_df['BeforePeakColor'] = all_df['BeforePeak'].apply(color_chooser)
all_data = ColumnDataSource(data=all_df)

# A data of the background points being plotted
plot_data = ColumnDataSource(data=all_df.copy())

# Holder for data defining the target entry
me_data = data.drop(data.index)
me_data = ColumnDataSource(data=me_data)

Define the plots

In [4]:
plot = Figure(plot_width=800, plot_height=400, x_axis_type='log')

background_plot = plot.scatter('AgingTime', 'AgingTemp', source=plot_data, color='BeforePeakColor', size=5)
plot.scatter('AgingTime', 'AgingTemp', source=me_data, marker='x', color='blue', size=20)

Make it pretty

In [5]:
plot.xaxis[0].axis_label = 'Aging Time (min)'
plot.yaxis[0].axis_label = 'Aging Temp. (C)'

Define the controls

In [6]:
update_button = Button(label='Update', button_type="success", name="target_button")
composition_slider = Slider(start=0, end=10, value=10, callback_throttle=100, title='Composition Tolerance')

Add some interaction tools

In [7]:
tool_tips = [
    ('Title', '@EntryTitle'),
    ('Composition', '@NominalComposition'),
    ('BeforePeak', '@BeforePeakLabel')
]
plot.add_tools(HoverTool(tooltips=tool_tips))

Create and attach the callback functions

In [13]:
# Function to change the target entry after querying form to find the current entry
update_callback = CustomJS(
    args=dict(
        plot=plot,
        all_data_obj=all_data, plot_data_obj=plot_data, me_data_obj=me_data,
        comp_tol=composition_slider),
    code="""
    // Get the datasets
    var all_data = all_data_obj.get('data');
    var target_data = me_data_obj.get('data');
    var plot_data = plot_data_obj.get('data');

    // Get the data from the variable
    var x = all_data['EntryTitle'].indexOf($("#current-entry").text());
    target_data['AgingTime'] = [all_data['AgingTime'][x]];
    target_data['AgingTemp'] = [all_data['AgingTemp'][x]];
    
    // Get entries within the tolerance of the Nb% of this entry
    var nb_tol = comp_tol.get('value');
    var nb_target = all_data['NominalNbPercent'][x];
    
    //  Find which entries pass the filter
    hits = []
    comp_dist = []
    for (var i=0; i<all_data['NominalNbPercent'].length; i++) {
        if (Math.abs(nb_target - all_data['NominalNbPercent'][i]) <= nb_tol) {    
            hits.push(i)
        }
    }
    
    //   Shrink plot_data
    for (var k in plot_data) {
        temp = []
        for (var i=0; i<hits.length; i++) {
            temp.push(all_data[k][hits[i]])
        }
        plot_data[k] = temp
    }
    
    // Trigger the plot to update
    var nb_min = Math.round(Math.max(0, nb_target - nb_tol) * 10) / 10
    var nb_max = Math.round(Math.min(100, nb_target + nb_tol) * 10) / 10
    plot.title.text = 'Values between U-' + nb_min + 'Nb and U-' + nb_max + 'Nb'
    plot.trigger('change');
    me_data_obj.trigger('change');
    plot_data_obj.trigger('change');
""")
update_button.callback = update_callback
composition_slider.callback = update_callback

### Packaging the Visualization
Turn the visualization into HTML and JS

In [9]:
layout = column(update_button, plot, composition_slider)
scripts, div = components(layout)

Put it in the correct output container

In [10]:
pinyon = {
    'scripts': scripts,
    'div': div,
    'button_id': 'modelid_' + update_button.ref['id'],
}

### See the Plot

In [11]:
show(layout)

## Output Cell (programmatically generated)

These commands connect to the Pwerwall server and send all desired output results back to the server

In [12]:
pickle.dumps(pinyon)

'(dp1\nS\'div\'\np2\nV\\u000a<div class="bk-root">\\u000a    <div class="plotdiv" id="8346ca77-78e0-4ff9-89a8-2f6db5db3279"></div>\\u000a</div>\np3\nsS\'button_id\'\np4\nS\'modelid_ad7c1910-1d02-45b3-8b55-bec2e8214df3\'\np5\nsS\'scripts\'\np6\nS\'\\n<script type="text/javascript">\\n    Bokeh.$(function() {\\n    var docs_json = {"82ece1ce-8ce5-4b97-8fb3-8f803885b3b9":{"roots":{"references":[{"attributes":{"fill_color":{"value":"blue"},"line_color":{"value":"blue"},"size":{"units":"screen","value":20},"x":{"field":"AgingTime"},"y":{"field":"AgingTemp"}},"id":"5ae9329d-b886-415e-a6ec-30b65708c0e4","type":"X"},{"attributes":{"children":[{"id":"9ff7b2a1-62d9-4b14-856e-53fb805abe6b","type":"Slider"}]},"id":"4bb9f071-2822-4266-bcbf-0f6b77f0c607","type":"WidgetBox"},{"attributes":{"plot":{"id":"67b5c4e9-2c3b-41fa-a84a-0a968ec4a779","subtype":"Figure","type":"Plot"}},"id":"48f68792-4672-44d8-b70b-9020342c4004","type":"WheelZoomTool"},{"attributes":{"callback":{"id":"62b5c929-fae5-468c-88ae-46